<a href="https://colab.research.google.com/github/jtao22/PythonAI/blob/main/GenrePredictions/spotifygenreprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import seaborn as sb
from sklearn.preprocessing import LabelEncoder as le
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import MinMaxScaler as mms
from sklearn.preprocessing import StandardScaler as ss
from sklearn.linear_model import LogisticRegression as lr
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.metrics import confusion_matrix as cm

In [ ]:
#Upload data
from google.colab import files 
files.upload()

In [8]:
data = pd.read_csv('songs.csv')
#filter
data = data.drop(["title","Unnamed: 0","song_name","analysis_url","track_href","uri","id","type"],axis=1)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#check
data.head(5)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre
0,0.831,0.814,2,-7.364,1,0.4200,0.0598,0.013400,0.0556,0.3890,156.985,124539,4,Dark Trap
1,0.719,0.493,8,-7.230,1,0.0794,0.4010,0.000000,0.1180,0.1240,115.080,224427,4,Dark Trap
2,0.850,0.893,5,-4.783,1,0.0623,0.0138,0.000004,0.3720,0.0391,218.050,98821,4,Dark Trap
3,0.476,0.781,0,-4.710,1,0.1030,0.0237,0.000000,0.1140,0.1750,186.948,123661,3,Dark Trap
4,0.798,0.624,2,-7.668,1,0.2930,0.2170,0.000000,0.1660,0.5910,147.988,123298,4,Dark Trap


In [34]:
#split data
trainX,testX,trainY,testY = tts(data.iloc[:,0:12].values, data.iloc[:,-1].values, test_size = 0.2, random_state = 42)
#scale features
scaler = mms(feature_range = (0,1))
trainX = scaler.fit_transform(trainX)
testX = scaler.fit_transform(testX)
# one-hot encoding
LE = le()
trainY = LE.fit_transform(trainY)
testY = LE.fit_transform(testY)

In [45]:
#create architecture function
def arch(trainX, trainY):
  #Logistic Regression
  logReg = lr(random_state = 42)
  logReg.fit(trainX, trainY)
  #Decision Tree
  DTC = dtc(criterion='entropy', random_state = 42)
  DTC.fit(trainX, trainY)
  #Random Forest Classifier
  RFC = rfc(n_estimators = 20, criterion = 'entropy', random_state = 42)
  RFC.fit(trainX, trainY)
  #print
  print('Logistic Regression Training Accuracy: ', logReg.score(trainX,trainY) * 100, '%')
  print('Decision Tree Classifier Accuracy: ', DTC.score(trainX,trainY)*100, '%')
  print('Random Forest Classifier Accuracy: ', RFC.score(trainX,trainY)*100, '%') 
  #return
  return logReg, DTC, RFC

In [46]:
#create model
models = arch(trainX, trainY)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Logistic Regression Training Accuracy:  57.72662805814915 %
Decision Tree Classifier Accuracy:  93.8718827561754 %
Random Forest Classifier Accuracy:  93.77733128471813 %


In [47]:
#test data
names = ['Logistic Regression', 'Decision Tree Classifier', 'Random Forest Classifier']
for i in range(3):
  predictions = models[i].predict(testX)
  count = 0
  total = len(predictions)
  for j in range(len(predictions)):
    if (predictions[j] == testY[j]):
      count = count + 1
  print(names[i],": ", count/total*100, '%')

Logistic Regression :  57.191821297718946 %
Decision Tree Classifier :  46.897529842808176 %
Random Forest Classifier :  60.90296655241697 %
